In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1

!git clone --recursive https://github.com/nvlabs/tiny-cuda-nn
%cd /content/tiny-cuda-nn
!cmake . -B build
!cmake --build build --config RelWithDebInfo -j
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

!git clone -b dev https://github.com/camenduru/CoDeF

!pip install -q pytorch_lightning einops ipdb torch_optimizer kornia scikit-video
# !pip install -q https://github.com/camenduru/wheels/releases/download/colab/tinycudann-1.7-cp310-cp310-linux_x86_64.whl



/content
env: TF_CPP_MIN_LOG_LEVEL=1
Cloning into 'tiny-cuda-nn'...
remote: Enumerating objects: 4082, done.
remote: Counting objects: 100% (1288/1288), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 4082 (delta 1114), reused 1149 (delta 1077), pack-reused 2794
Receiving objects: 100% (4082/4082), 19.60 MiB | 33.07 MiB/s, done.
Resolving deltas: 100% (2623/2623), done.
Submodule 'dependencies/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'dependencies/cutlass'
Submodule 'dependencies/fmt' (https://github.com/fmtlib/fmt) registered for path 'dependencies/fmt'
Cloning into '/content/tiny-cuda-nn/dependencies/cutlass'...
remote: Enumerating objects: 26708, done.        
remote: Counting objects: 100% (20/20), done.        
remote: Compressing objects: 100% (18/18), done.        
remote: Total 26708 (delta 4), reused 10 (delta 0), pack-reused 26688        
Receiving objects: 100% (26708/26708), 42.65 MiB | 29.57 MiB/s, done.
Resolving deltas: 100%

In [3]:
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/all_sequences.zip -d /content/tiny-cuda-nn/CoDeF/ckpts -o all_sequences.zip
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/beauty_0.ckpt -d /content/tiny-cuda-nn/CoDeF/ckpts -o beauty_0.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/beauty_1.ckpt -d /content/tiny-cuda-nn/CoDeF/ckpts -o beauty_1.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/lemon_hit.ckpt -d /content/tiny-cuda-nn/CoDeF/ckpts -o lemon_hit.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/scene_0.ckpt -d /content/tiny-cuda-nn/CoDeF/ckpts -o scene_0.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/white_smoke.ckpt -d /content/tiny-cuda-nn/CoDeF/ckpts -o white_smoke.ckpt

!unzip /content/tiny-cuda-nn/CoDeF/ckpts/all_sequences.zip -d /content/tiny-cuda-nn/CoDeF/ckpts
!rm -rf /content/tiny-cuda-nn/CoDeF/ckpts/__MACOSX

# !mkdir /content/CoDeF/ckpts/all_sequences/scene_0/base
# !cp /content/CoDeF/ckpts/scene_0.ckpt /content/CoDeF/ckpts/all_sequences/scene_0/base/scene_0.ckpt

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) 

In [4]:
import shutil

source_dir = '/content/tiny-cuda-nn/CoDeF/ckpts/all_sequences/'
destination_dir = '/content/tiny-cuda-nn/CoDeF/all_sequences/'
shutil.copytree(source_dir, destination_dir)

'/content/tiny-cuda-nn/CoDeF/all_sequences/'

In [5]:
import os
current_directory = os.getcwd()
print("Current directory:", current_directory)

Current directory: /content/tiny-cuda-nn


In [6]:
%cd /content/tiny-cuda-nn/CoDeF/

/content/tiny-cuda-nn/CoDeF


In [10]:
source_dir = '/content/drive/MyDrive/blackswan'
destination_dir = '/content/tiny-cuda-nn/CoDeF/all_sequences/blackswan'
shutil.copytree(source_dir, destination_dir)


'/content/tiny-cuda-nn/CoDeF/all_sequences/blackswan'

In [11]:
!pwd

/content/tiny-cuda-nn/CoDeF


In [15]:
# @title Train (Step 1)

GPUS=0
NAME="blackswan"
EXP_NAME="base"
ROOT_DIRECTORY=f"all_sequences/{NAME}/{NAME}"
MODEL_SAVE_PATH=f"ckpts/all_sequences/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/{NAME}.ckpt"
# WEIGHT_PATH=f"ckpts/{NAME}.ckpt"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
# MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks"
FLOW_DIRECTORY=f"all_sequences/{NAME}/{NAME}_flow"

!python train.py --root_dir $ROOT_DIRECTORY \
                --model_save_path $MODEL_SAVE_PATH \
                --log_save_path $LOG_SAVE_PATH  \
                --gpus $GPUS \
                --encode_w --annealed \
                --config $CONFIG_DIRECTORY \
                --exp_name $EXP_NAME
                # --mask_dir $MASK_DIRECTORY
                # --flow_dir $FLOW_DIRECTORY

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

Missing logger folder: logs/test_all_sequences/blackswan/base
2024-05-25 09:40:35.146414: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has alread

In [14]:
# @title Generate Canonical Image (Step 2)

GPUS=0
NAME="blackswan"
EXP_NAME="base"
ROOT_DIRECTORY=f"all_sequences/{NAME}/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/last.ckpt"
# WEIGHT_PATH=f"ckpts/{NAME}.ckpt"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"

!python train.py  --test --encode_w \
                  --root_dir $ROOT_DIRECTORY \
                  --log_save_path $LOG_SAVE_PATH \
                  --mask_dir $MASK_DIRECTORY \
                  --weight_path $WEIGHT_PATH \
                  --gpus $GPUS \
                  --config $CONFIG_DIRECTORY \
                  --exp_name $EXP_NAME \
                  --save_deform False

Traceback (most recent call last):
  File "/content/tiny-cuda-nn/CoDeF/train.py", line 557, in <module>
    main(hparams)
  File "/content/tiny-cuda-nn/CoDeF/train.py", line 519, in main
    system = ImplicitVideoSystem(hparams)
  File "/content/tiny-cuda-nn/CoDeF/train.py", line 99, in __init__
    load_ckpt(embedding_w, hparams.weight_path, model_name=f'w_{i}')
  File "/content/tiny-cuda-nn/CoDeF/utils/__init__.py", line 98, in load_ckpt
    checkpoint_ = extract_model_state_dict(ckpt_path, model_name, prefixes_to_ignore)
  File "/content/tiny-cuda-nn/CoDeF/utils/__init__.py", line 78, in extract_model_state_dict
    checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 997, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 444, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/loca

In [3]:
# @title Generate Video (Step 3)

GPUS=0
NAME="blackswan"
EXP_NAME="base"
ROOT_DIRECTORY=f"all_sequences/{NAME}/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
# CANONICAL_DIR=f"ckpts/all_sequences/{NAME}/{EXP_NAME}_control"
CANONICAL_DIR=f"/content/tiny-cuda-nn/CoDeF/all_sequences/{NAME}/base_control"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/last.ckpt"
# WEIGHT_PATH=f"ckpts/{NAME}.ckpt"

!python train.py --test --encode_w \
                --root_dir $ROOT_DIRECTORY \
                --log_save_path $LOG_SAVE_PATH \
                --weight_path $WEIGHT_PATH \
                --gpus $GPUS \
                --canonical_dir $CANONICAL_DIR \
                --config $CONFIG_DIRECTORY \
                --exp_name $EXP_NAME

python3: can't open file '/content/train.py': [Errno 2] No such file or directory
